In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/ImageAndXML_Data-20220716T104126Z-001.zip

Streaming output truncated to the last 5000 lines.
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_29.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_40.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_31.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_28.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_56.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_32.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_45.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_46.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_21.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_33.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_38.tif  
  inflating: ImageAndXML_Data/505852048_505852049/505852048_505852049_26.tif  
 

In [ ]:
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import pandas as pd
import tqdm

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

In [ ]:
import tensorflow as tf

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5302914163201261252
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16139419648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11956191321864937324
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [ ]:
char_list = string.ascii_letters+string.digits
 
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            a = char
#             print(char)
        
    return dig_lst

In [ ]:
def prepocessing_image(img_path):
    # read input image and convert into gray scale image
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2GRAY)   

    # convert each image of shape (32, 128, 1)
    w, h = img.shape
    if h > 128 or w > 32:
#         continue
        return img,True
    if w < 32:
        add_zeros = np.ones((32-w, h))*255
        img = np.concatenate((img, add_zeros))

    if h < 128:
        add_zeros = np.ones((32, 128-h))*255
        img = np.concatenate((img, add_zeros), axis=1)
    
#     w2,h2 = img.shape
#     print(h2,w2)
    img = np.expand_dims(img , axis = 2)
    

    # Normalize each image
    img = img/255.
    return img,False

In [ ]:
def generate_dataset(train_path):
  dataset_img = []
  dataset_txt = []
  dataset_input_length = []
  dataset_label_length = []
  dataset_orig_txt = []

  max_label_len = 0

  for root, dirnames, filenames in os.walk(train_path):
  
      for f_name in fnmatch.filter(filenames, '*.csv'):
          folder_name_arr = f_name.split('.')[:-1]
          folder_name = '.'.join(folder_name_arr)
  #         print(folder_name_arr)
          df = pd.read_csv(train_path+'/'+f_name, sep=",")
          image_df_list = df.values.tolist()
          for file in image_df_list:
              file_path = train_path+'/'+folder_name+'/'+file[1]
              file_label = file[2]
              img,is_skip = prepocessing_image(file_path)
              if is_skip:
  #                 print(file_label)
                  continue
              
  #             print(file_label)

              encoded_label = encode_to_labels(file_label)
              # compute maximum length of the text
              if len(encoded_label) > max_label_len:
                  max_label_len = len(encoded_label)
              
              if len(encoded_label) == 0:
                  # print(file_label,len(encoded_label))
                  a = len(file_label)
              else:
                # dataset_orig_txt.append(file_label)
                dataset_orig_txt.append(file_label)     
                dataset_label_length.append(len(encoded_label))
                dataset_input_length.append(31)
                dataset_img.append(img)
                # dataset_txt.append(file_label)

                dataset_txt.append(encode_to_labels(file_label))

  # tokenizer = Tokenizer(num_words = 200, oov_token = "<OOV>",char_level=True)
  # tokenizer.fit_on_texts(dataset_txt)
  # dataset_txt = tokenizer.texts_to_sequences(dataset_txt)
  # # print(sequences)
  # max_label_len = 0
  # for i in dataset_txt:
  #     if len(i) > max_label_len:
  #         max_label_len = len(i)
  #     if len(i) > 0:   
  #       dataset_label_length.append(len(i))
  #       dataset_input_length.append(31)
  #     else:
  #       print(i)
  #       index = dataset_txt.index(i)
  #       del dataset_img[index]
  #       del dataset_orig_txt[index]
  #       del dataset_txt[index]

  # char_list = tokenizer.word_index

  num_train = len(dataset_txt)*75//100
  # print(num_valid)
  #lists for validation dataset
  valid_img = dataset_img[num_train:-1]
  valid_txt = dataset_txt[num_train:-1]
  valid_input_length = dataset_input_length[num_train:-1]
  valid_label_length = dataset_label_length[num_train:-1]
  valid_orig_txt = dataset_orig_txt[num_train:-1]


  training_img = dataset_img[0:num_train]
  training_txt = dataset_txt[0:num_train]
  train_input_length = dataset_input_length[0:num_train]
  train_label_length = dataset_label_length[0:num_train]
  orig_txt = dataset_orig_txt[0:num_train]

  train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
  valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

  training_img = np.array(training_img)
  train_input_length = np.array(train_input_length)
  train_label_length = np.array(train_label_length)

  valid_img = np.array(valid_img)
  valid_input_length = np.array(valid_input_length)
  valid_label_length = np.array(valid_label_length)

  return training_img,training_txt,train_input_length,train_label_length,orig_txt,train_padded_txt,valid_img,valid_txt,valid_input_length,valid_label_length,valid_orig_txt,valid_padded_txt,max_label_len


In [ ]:
def generate_model(img_shape,char_length):
  # input with shape of height=32 and width=128 
  inputs = Input(shape=(32,128,1))
  
  # convolution layer with kernel size (3,3)
  conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
  # poolig layer with kernel size (2,2)
  pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
  
  conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
  pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
  
  conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
  
  conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
  # poolig layer with kernel size (2,1)
  pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
  
  conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
  # Batch normalization layer
  batch_norm_5 = BatchNormalization()(conv_5)
  
  conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
  batch_norm_6 = BatchNormalization()(conv_6)
  pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
  
  conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
  
  squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
  
  # bidirectional LSTM layers with units=128
  blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
  blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
  
  outputs = Dense(char_length+1, activation = 'softmax')(blstm_2)

  return inputs,outputs


In [ ]:
def train():
  img_shape = (32,128,1)
  batch_size = 256
  epochs = 30
  train_path = 'ImageAndXML_Data'
  file_model ="test.hdf5"
  training_img,training_txt,train_input_length,train_label_length,orig_txt,train_padded_txt,valid_img,valid_txt,valid_input_length,valid_label_length,valid_orig_txt,valid_padded_txt,max_label_len = generate_dataset(train_path)

  # # save char_list to csv
  # df = pd.DataFrame(char_list, index=[0])
  # df.to_csv('char_list.csv', index=False)

  labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
  input_length = Input(name='input_length', shape=[1], dtype='int64')
  label_length = Input(name='label_length', shape=[1], dtype='int64')


  def ctc_lambda_func(args):
      y_pred, labels, input_length, label_length = args
      return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


  inputs,outputs = generate_model(img_shape,len(char_list))

  loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

  #model to be used at training time
  model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)
  model.summary()
  model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam',metrics=["accuracy"])

  checkpoint = ModelCheckpoint(filepath=file_model, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
  callbacks_list = [checkpoint]
  model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], y=np.zeros(len(training_img)), batch_size=batch_size, epochs = epochs, validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]), verbose = 1, callbacks = callbacks_list)

In [ ]:
train()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 128, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_70 (Conv2D)             (None, 32, 128, 64)  640         ['input_11[0][0]']               
                                                                                                  
 max_pooling2d_40 (MaxPooling2D  (None, 16, 64, 64)  0           ['conv2d_70[0][0]']              
 )                                                                                                
                                                                                           

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



Epoch 1: val_loss improved from inf to 31.38420, saving model to test.hdf5
11288/11288 [==============================] - 30s 3ms/sample - loss: 34.5589 - acc: 0.0000e+00 - val_loss: 31.3842 - val_acc: 0.0000e+00
Epoch 2/30
11288/11288 [==============================] - ETA: 0s - loss: 29.7172 - acc: 0.0000e+00
Epoch 2: val_loss improved from 31.38420 to 30.84142, saving model to test.hdf5
11288/11288 [==============================] - 24s 2ms/sample - loss: 29.7172 - acc: 0.0000e+00 - val_loss: 30.8414 - val_acc: 0.0000e+00
Epoch 3/30
11288/11288 [==============================] - ETA: 0s - loss: 28.3783 - acc: 0.0000e+00
Epoch 3: val_loss did not improve from 30.84142
11288/11288 [==============================] - 24s 2ms/sample - loss: 28.3783 - acc: 0.0000e+00 - val_loss: 37.9906 - val_acc: 0.0000e+00
Epoch 4/30
11288/11288 [==============================] - ETA: 0s - loss: 26.2403 - acc: 0.0000e+00
Epoch 4: val_loss did not improve from 30.84142
11288/11288 [=====================

In [ ]:
# def predict(img_path){
img_shape = (32,128,1)
inputs,outputs = generate_model(img_shape,len(char_list))
model = Model(inputs,outputs)
# load the saved best model weights
model.load_weights('test.hdf5')
model.summary()

train_path = 'ImageAndXML_Data'

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 32, 128, 1)]      0         
                                                                 
 conv2d_63 (Conv2D)          (None, 32, 128, 64)       640       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 16, 64, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_64 (Conv2D)          (None, 16, 64, 128)       73856     
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 8, 32, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          (None, 8, 32, 256)        2951

In [ ]:
valid_img[0].shape


(32, 128, 1)